In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, GPT2Model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from bayes_opt import BayesianOptimization
import random
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

2024-08-12 14:46:48.817083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 14:46:48.817186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 14:46:48.948649: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# set the random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [4]:
# Load the dataset
dataset = load_dataset("dair-ai/emotion", "split")

# Load each split
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

def stratified_sample(dataset, fraction):
    label_counts = defaultdict(list)
    
    # Group indices by label
    for i, example in enumerate(dataset):
        label_counts[example['label']].append(i)
    
    sampled_indices = []
    for label, indices in label_counts.items():
        # Calculate the number of samples to draw for each label
        sample_size = int(len(indices) * fraction)
        sampled_indices.extend(random.sample(indices, min(sample_size, len(indices))))
    
    return dataset.select(sampled_indices)

# Calculate the fraction size (one fifth) for each dataset
fraction = 1 / 2

# Perform stratified sampling
sampled_train_dataset = stratified_sample(train_dataset, fraction)
sampled_val_dataset = stratified_sample(val_dataset, fraction)
sampled_test_dataset = stratified_sample(test_dataset, fraction)

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
# Load GPT2Tokenizer and Model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')



# Add pad_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2ForSequenceClassification.from_pretrained('distilgpt2', num_labels=6)

# Resize token embeddings to match the tokenizer length
model.resize_token_embeddings(len(tokenizer))

# Set the padding token ID in the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
compareprompts = [
"What mood does the author convey in the message?",
"What would be your feelings towards the message?",
"How would you describe the author's feeling in the message?"
]

In [7]:
# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments setup
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",  # Ensure reproducibility
    seed=42
)

# Data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluate each prompt from compareprompts
for prompt in compareprompts:
    print(f"Evaluating with prompt: {prompt}")

    # Function to tokenize text with the given prompt
    def tokenize_with_prompt(examples):
        inputs = [f"{prompt} {text}" for text in examples['text']]
        return tokenizer(inputs, truncation=True, padding='max_length', max_length=512)

    # Apply the tokenize function to datasets
    train_data_with_prompt = sampled_train_dataset.map(tokenize_with_prompt, batched=True)
    val_data_with_prompt = sampled_val_dataset.map(tokenize_with_prompt, batched=True)
    test_data_with_prompt = sampled_test_dataset.map(tokenize_with_prompt, batched=True)

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_with_prompt,
        eval_dataset=val_data_with_prompt,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model on the test set
    eval_result = trainer.evaluate(test_data_with_prompt)
    print(f"Final evaluation results for prompt '{prompt}' on test set: {eval_result}")

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Evaluating with prompt: What mood does the author convey in the message?


Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.138900,0.210999,0.930931,0.931037,0.932174,0.930931
2,0.116600,0.221449,0.923924,0.923472,0.924764,0.923924
3,0.091800,0.273471,0.929930,0.928887,0.929326,0.929930
4,0.092100,0.267106,0.924925,0.925070,0.926143,0.924925
5,0.070800,0.272114,0.932933,0.932811,0.933256,0.932933
6,0.063900,0.244838,0.929930,0.929565,0.929975,0.929930
7,0.057800,0.272062,0.924925,0.924440,0.924630,0.924925
8,0.045600,0.272488,0.932933,0.932268,0.932894,0.932933
9,0.036500,0.272679,0.936937,0.936124,0.936711,0.936937
10,0.033700,0.274472,0.932933,0.932413,0.932804,0.932933


Final evaluation results for prompt 'How would you describe the author's feeling in the message?' on test set: {'eval_loss': 0.2148037552833557, 'eval_accuracy': 0.9198396793587175, 'eval_f1': 0.9188766298425116, 'eval_precision': 0.9199351286206492, 'eval_recall': 0.9198396793587175, 'eval_runtime': 10.1156, 'eval_samples_per_second': 98.659, 'eval_steps_per_second': 6.228, 'epoch': 10.0}
